In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import joblib
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report



In [2]:

# Load models and scaler
cnn_model = tf.keras.models.load_model('trained_model.keras')
scaler = joblib.load('scaler.pkl')
svm_model = joblib.load('svm_wildfire_model.pkl')


In [3]:

# Email alert function -smpt(simple mail transfer protocol)
def send_email_alert(subject, body):
    sender_email = "ayevani.2004@gmail.com"
    receiver_email = "pratrika10@gmail.com"
    password = "bxxv khxi dfvt xgsc"

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(sender_email, password)
        server.send_message(msg)
        server.quit()
        print("Email alert sent!")
    except Exception as e:
        print(f"Error: {e}")



In [4]:
# GUI application
def browse_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    image_path_var.set(file_path)



In [5]:
def predict():
    try:
        # Validate Image Path
        image_path = image_path_var.get()
        if not image_path:
            messagebox.showerror("Error", "Please select an image.")
            return

        # CNN Prediction
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(256, 256))
        input_arr = tf.keras.preprocessing.image.img_to_array(image)
        input_arr = np.array([input_arr])  # Convert to batch format
        cnn_prediction = cnn_model.predict(input_arr)
        cnn_score = np.argmax(cnn_prediction)  # 1 for 'fire', 0 for 'nofire'

        # Validate Sensor Data
        try:
            sensor_data = [
                float(sensor_ndvi.get()), 
                float(sensor_lst.get()), 
                float(sensor_humidity.get())
            ]
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values for sensor data.")
            return

        # SVM Prediction
        input_df = pd.DataFrame([sensor_data], columns=['NDVI', 'LST', 'HUMIDITY'])
        input_scaled = scaler.transform(input_df)
        svm_prediction = svm_model.predict(input_scaled)
        svm_score = 1 if svm_prediction[0] == 'fire' else 0  # Convert to numeric score

        # Combined Decision
        cnn_weight = 0.7
        svm_weight = 0.3
        combined_score = (cnn_score * cnn_weight) + (svm_score * svm_weight)

        # Display Results and Send Email
        if combined_score >= 0.5:
            result.set("Fire detected!")
            send_email_alert("Fire Alert", "Fire detected based on combined analysis of satellite and sensor data!")
        else:
            result.set("No fire detected.")
            send_email_alert("No Fire Alert", "No fire detected based on combined analysis of satellite and sensor data.")

        # Debugging Outputs
        print(f"Raw CNN Prediction: {cnn_prediction}")
        print(f"CNN Model Prediction (Class): {'fire' if cnn_score == 1 else 'nofire'}")
        print(f"Raw SVM Prediction: {svm_prediction}")
        print(f"CNN Score: {cnn_score}, SVM Score: {svm_score}")
        print(f"Combined Weighted Score: {combined_score}")

    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")


In [6]:
# Create GUI window
app = tk.Tk()
app.title("Wildfire Detection System")
app.geometry("500x400")



''

In [7]:
# GUI elements
image_path_var = tk.StringVar()
sensor_ndvi = tk.StringVar()
sensor_lst = tk.StringVar()
sensor_humidity = tk.StringVar()
result = tk.StringVar()



In [8]:
# Labels and entries
tk.Label(app, text="Wildfire Detection System", font=("Arial", 16)).pack(pady=10)


In [9]:

# Image input
tk.Label(app, text="Select Satellite Image:").pack(anchor="w", padx=20)
image_entry = tk.Entry(app, textvariable=image_path_var, width=40)
image_entry.pack(anchor="w", padx=20)
tk.Button(app, text="Browse", command=browse_image).pack(anchor="w", padx=20, pady=5)




In [10]:
# Sensor data input
tk.Label(app, text="Enter Sensor Data:").pack(anchor="w", padx=20, pady=10)

frame = tk.Frame(app)
frame.pack(anchor="w", padx=20)

tk.Label(frame, text="NDVI:").grid(row=0, column=0, sticky="w")
tk.Entry(frame, textvariable=sensor_ndvi).grid(row=0, column=1)

tk.Label(frame, text="LST:").grid(row=1, column=0, sticky="w")
tk.Entry(frame, textvariable=sensor_lst).grid(row=1, column=1)

tk.Label(frame, text="Humidity:").grid(row=2, column=0, sticky="w")
tk.Entry(frame, textvariable=sensor_humidity).grid(row=2, column=1)



In [11]:
# Predict button
tk.Button(app, text="Predict", command=predict).pack(pady=20)



In [12]:
# Result display
tk.Label(app, text="Result:").pack()
result_label = tk.Label(app, textvariable=result, font=("Arial", 14))
result_label.pack(pady=10)



In [ ]:
# Run the GUI
app.mainloop()

1/1 [==============================] - 0s 134ms/step
Email alert sent!
Raw CNN Prediction: [[0.0533987 0.9466013]]
CNN Model Prediction (Class): fire
Raw SVM Prediction: ['fire']
CNN Score: 1, SVM Score: 1
Combined Weighted Score: 1.0
1/1 [==============================] - 0s 153ms/step
Email alert sent!
Raw CNN Prediction: [[0.86472964 0.13527037]]
CNN Model Prediction (Class): nofire
Raw SVM Prediction: ['fire']
CNN Score: 0, SVM Score: 1
Combined Weighted Score: 0.3
1/1 [==============================] - 0s 165ms/step
Email alert sent!
Raw CNN Prediction: [[0.86472964 0.13527037]]
CNN Model Prediction (Class): nofire
Raw SVM Prediction: ['fire']
CNN Score: 0, SVM Score: 1
Combined Weighted Score: 0.3
1/1 [==============================] - 0s 158ms/step
Email alert sent!
Raw CNN Prediction: [[0.86472964 0.13527037]]
CNN Model Prediction (Class): nofire
Raw SVM Prediction: ['fire']
CNN Score: 0, SVM Score: 1
Combined Weighted Score: 0.3
1/1 [==============================] - 0s 270ms/